In [9]:
from package.model import DiffusionModel
from package.topic import TopicModel
from package.social_graph import SN_Graph
from package.cluster_graph import ClusterGraph
from package.itemset import ItemsetFlyweight, ItemRelation
from package.utils import getItemsPrice, read_items
from package.algorithm import Algorithm
from package.coupon import Coupon
from package.user_proxy import UsersProxy
from package.itemset import Itemset

import pandas as pd
import networkx as nx

In [10]:
TOPICS = {
    "Cluster": {
        '1': [0.4139, 0.2764, 0.3097],
        '2, 3, 5': [0.31980000000000003, 0.31626666666666664, 0.3639333333333333],
        '6': [0.0213, 0.5398, 0.4389],
        '0, 9': [0.11779999999999999, 0.47209999999999996, 0.4101],
        '1, 3': [0.3639, 0.2671, 0.369],
        '4': [0.4535, 0.5131, 0.0334],
        '8': [0.0816, 0.0634, 0.855],
        '3': [0.3139, 0.2578, 0.4283]

    },
    "Item": {
        "iPhone": [0.7, 0.0, 0.3],
        "AirPods": [0.9, 0.0, 0.1],
        "Galaxy": [0.0, 0.8, 0.2],
    }
}
PRICES = {
    "iPhone": 260,
    "AirPods": 60,
    "Galaxy": 500,
}
RELATION = pd.DataFrame.from_dict({
            "iPhone":{
                "AirPods":10,
                "Galaxy":-5
            },
            "AirPods":{
                "iPhone":1,
                "Galaxy":0,
            },
            "Galaxy":{
                "iPhone":-8,
                "AirPods":1,
            }
            })

edges = {('1', '2, 3, 5'): 2.3333333333333335,
    ('1', '6'): 1.0,
    ('2, 3, 5', '0, 9'): 2.0,
    ('2, 3, 5', '1, 3'): 0.8333333333333333,
    ('2, 3, 5', '4'): 0.75,
    ('2, 3, 5', '8'): 0.75,
    ('0, 9', '1'): 0.5,
    ('1, 3', '2, 3, 5'): 2.3333333333333335,
    ('1, 3', '4'): 0.5,
    ('1, 3', '6'): 1.0,
    ('4', '3'): 0.3333333333333333,
    ('3', '4'): 0.5}

relation = ItemRelation(RELATION)
topic = TopicModel(3, TOPICS["Cluster"], TOPICS["Item"])
_itemset = ItemsetFlyweight(PRICES, topic, relation)
_graph = ClusterGraph(cluster_topic=topic.getNodesTopic(), theta = 0.9, depth = 5)
_user_proxy = UsersProxy(_graph, _itemset, [])
_graph.add_weighted_edges_from([(u, v, edges[u, v]) for u, v in edges.keys()])

In [16]:
node = "2, 3, 5"
result = _user_proxy.getVPsByUserId(node)
result = sorted(result.items(), key= lambda item:item[1],reverse=True)
print("{}'s CP".format(node))
for k, v in result:
    print("{}: {}".format(k,v))
print()


2, 3, 5's CP
AirPods: 0.0054035555555555555
iPhone: 0.0012809230769230769
AirPods iPhone: 0.001026085438584013
Galaxy: 0.0006516
AirPods Galaxy: 0.0005803573492964911
Galaxy iPhone: 0.00043332771171423047
AirPods Galaxy iPhone: 0.0003994055542775261



In [12]:
discountable = list(str(obj) for obj in _user_proxy.discoutableItems(node, _itemset["Galaxy"]))
print(discountable)
print(_user_proxy._VP_ratio(node, "Galaxy"))
for ids in discountable:
    print(_user_proxy._min_discount(node, "Galaxy", ids), _user_proxy._VP_ratio(node, ids))

[]
0.0010392399999999999


In [13]:
import math
min_discount = math.ceil(_user_proxy._min_discount(node, "Galaxy", "Galaxy iPhone"))
min_discount

435

In [19]:
_graph.edges['1', '2, 3, 5']

{'weight': 0.5, 'is_tested': False}

In [17]:

coupon = Coupon(accThreshold=_itemset["Galaxy"].price+61, 
                accItemset=_itemset["Galaxy"], 
                discount=min_discount,
                disItemset=_itemset["iPhone"])

_graph.nodes[node]["desired_set"] = _itemset["Galaxy AirPods"]
result = _user_proxy.getVPsByUserId(node)
result = sorted(result.items(), key= lambda item:item[1],reverse=True)
print("{}'s CP".format(node))
for k, v in result:
    print("{}: {}".format(k,v))
print()



2, 3, 5's CP
AirPods: 0.0054035555555555555
iPhone: 0.0012809230769230769
AirPods iPhone: 0.001026085438584013
Galaxy: 0.0006516
AirPods Galaxy: 0.0005803573492964911
Galaxy iPhone: 0.00043332771171423047
AirPods Galaxy iPhone: 0.0003994055542775261

